In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
import matplotlib.pyplot as plt

# functions

In [3]:
def extractz500_several_files(filestemp):
    listxarrays = []
    for file in filestemp:
        dstemp = xr.open_dataset(file)
        dstemp = dstemp.sel(time=slice('1948-01-01', '2100-12-31'))
        dstemp = dstemp.TREFHTMX.where((dstemp.lat>=10),drop=True)
        # Transpose the data to match the desired dimension order
        dstemp_transposed = dstemp.transpose('time', 'lat', 'lon')
        lat0=20; lat1=80; lon0=180; lon1=330
        dstemp_transposed = dstemp_transposed.where((dstemp_transposed.lat>=lat0)&(dstemp_transposed.lat<=lat1)&\
                               (dstemp_transposed.lon>=lon0)&(dstemp_transposed.lon<=lon1),drop=True)
        
        # Recreate the DataArray with the coordinates in the desired order
        dstemp_reordered = xr.DataArray(
            dstemp_transposed.values,
            dims=['time', 'lat', 'lon'],
            coords={
                'time': dstemp_transposed.coords['time'],
                'lat': dstemp_transposed.coords['lat'],
                'lon': dstemp_transposed.coords['lon']
            },
            attrs=dstemp.attrs,
            name=dstemp.name
        )
        listxarrays.append(dstemp_reordered)
    xarrayfull = xr.concat(listxarrays, dim='time')
    xarrayfull = xarrayfull.drop_duplicates('time',keep='first')
    cftime_index = xr.coding.cftimeindex.CFTimeIndex(xarrayfull['time'].values)
    datetime_index = cftime_index.to_datetimeindex()
    xarrayfull.coords['time'] = datetime_index
    return xarrayfull.to_dataset()

# compute

In [9]:
path_temp_files = '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/day_1/TREFHTMX/'

In [10]:
filenames = np.sort(glob.glob(f'{path_temp_files}*'))

type_aerosols = []
for i in range(len(filenames)):
    if 'cmip6' in filenames[i].split('.')[2]:
        type_aerosols.append('cmip6')
    elif 'smbb' in filenames[i].split('.')[2]:
        type_aerosols.append('smbb')

In [11]:
path_outputs_higherthanp90 = '/glade/derecho/scratch/jhayron/Data4WRsClimateChange/LENS2_higherthanp90_v2/'

In [12]:
names_experiments_all_0 = np.array(['.'.join(filenames[i].split('.')[4:-5]) for i in range(len(filenames))])
names_experiments_all = []
for i in range(len(filenames)):
    names_experiments_all.append(f'{names_experiments_all_0[i]}_{type_aerosols[i]}')
names_experiments_all = np.array(names_experiments_all)
unique_names_experiments = np.unique(names_experiments_all)

In [15]:
def compute_higherthanp90(id_experiment):
    name_experiment = unique_names_experiments[id_experiment]
    print(name_experiment)
    where_files = np.where(names_experiments_all==name_experiment)[0]
    files_temp = filenames[where_files]
    
    dataset_raw = extractz500_several_files(files_temp)
    land_mask_file='/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/LANDFRAC/b.e21.BHISTcmip6.f09_g17.LE2-1001.001.cam.h0.LANDFRAC.185001-185912.nc'
    namevar='TREFHTMX'
    lsm = xr.open_dataset(land_mask_file).LANDFRAC[0]
    lsm = lsm.sel(lat=dataset_raw.lat,lon=dataset_raw.lon)
    lsm = lsm.astype(bool)
    mask_arr = lsm.data.astype(float)
    mask_arr[mask_arr==False]=np.nan
    data_arr = dataset_raw[namevar].data
    data_arr = data_arr*mask_arr
    dataset_raw[namevar].data = data_arr

    dataset_raw['year'] = dataset_raw['time.year']
    dataset_raw['dayofyear'] = ('time',np.tile(
            np.arange(1, 366, 1, dtype='int32'),
            len(pd.to_datetime(dataset_raw.time).year.unique())
        ))

    maxtemp_reshaped = dataset_raw.set_index(time=('year', 'dayofyear')).unstack('time')
    q90 = maxtemp_reshaped.sel(year=slice(1994,2023)).quantile(0.9,dim='year')
    
    stacked = xr.concat([q90, q90, q90], dim='dayofyear')
    stacked = stacked.rolling(dayofyear=5, center=True).mean()
    q90_smooth = stacked.isel(dayofyear=slice(365, 365*2))
    
    higherthanp90 = maxtemp_reshaped > q90_smooth
    higherthanp90 = higherthanp90.stack(time=('year', 'dayofyear'))
    higherthanp90 = higherthanp90.transpose('time', 'lat', 'lon').astype(int)
    
    higherthanp90 = higherthanp90.drop_vars(['time','year','dayofyear'])
    higherthanp90['time'] = dataset_raw['time']
    
    higherthanp90.to_netcdf(f"{path_outputs_higherthanp90}tmax_higherthanp90_{name_experiment}.nc")

In [16]:
from multiprocessing import Pool

num_ids = len(unique_names_experiments)
num_processors = 13

# Create a Pool of worker processes
with Pool(processes=num_processors) as pool:
    # Map the function to the range of IDs
    pool.map(compute_higherthanp90, range(num_ids))

LE2-1001.001_cmip6LE2-1041.003_cmip6LE2-1101.006_cmip6LE2-1141.008_cmip6LE2-1061.004_cmip6LE2-1021.002_cmip6LE2-1121.007_cmip6LE2-1081.005_cmip6LE2-1161.009_cmip6LE2-1181.010_cmip6LE2-1231.001_cmip6LE2-1231.003_cmip6

LE2-1231.005_cmip6












/glade/derecho/scratch/jhayron/tmp/ipykernel_20907/1346910074.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_20907/1346910074.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_20907/1346910074.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may

LE2-1151.008_smbb
LE2-1231.006_cmip6
LE2-1171.009_smbb
LE2-1131.007_smbb
LE2-1011.001_smbb
LE2-1091.005_smbb
LE2-1071.004_smbb
LE2-1231.004_cmip6
LE2-1111.006_smbb
LE2-1031.002_smbb
LE2-1051.003_smbb
LE2-1231.002_cmip6
LE2-1191.010_smbb


/glade/derecho/scratch/jhayron/tmp/ipykernel_20907/1346910074.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_20907/1346910074.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_20907/1346910074.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may

LE2-1231.007_cmip6
LE2-1231.009_cmip6
LE2-1231.011_smbb
LE2-1231.013_smbb
LE2-1231.015_smbb
LE2-1231.017_smbb
LE2-1231.019_smbb
LE2-1251.001_cmip6
LE2-1251.003_cmip6
LE2-1251.005_cmip6
LE2-1251.007_cmip6
LE2-1251.009_cmip6
LE2-1251.011_smbb


...

/glade/derecho/scratch/jhayron/tmp/ipykernel_20907/1346910074.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/work/jhayron/conda-envs/weather_regimes/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/glade/derecho/scratch/jhayron/tmp/ipykernel_20907/1346910074.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/work/jhayron/conda-envs/we

LE2-1281.019_smbb
LE2-1301.001_cmip6
LE2-1301.003_cmip6
LE2-1301.005_cmip6
LE2-1301.007_cmip6
LE2-1301.009_cmip6
LE2-1301.011_smbb
LE2-1301.013_smbb
LE2-1301.015_smbb
LE2-1301.017_smbb
LE2-1301.019_smbb


/glade/derecho/scratch/jhayron/tmp/ipykernel_20907/1346910074.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_20907/1346910074.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_20907/1346910074.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may

LE2-1301.002_cmip6
LE2-1301.004_cmip6
LE2-1281.020_smbb
LE2-1301.010_cmip6
LE2-1301.012_smbb
LE2-1301.014_smbb
LE2-1301.016_smbb
LE2-1301.018_smbb
LE2-1301.006_cmip6
LE2-1301.008_cmip6
LE2-1301.020_smbb


/glade/derecho/scratch/jhayron/tmp/ipykernel_20907/1346910074.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/derecho/scratch/jhayron/tmp/ipykernel_20907/1346910074.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()
/glade/work/jhayron/conda-envs/weather_regimes/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/glade/work/jhayron/conda-envs/we

In [66]:
15*12

180

In [70]:
13*8

104

In [14]:
id_experiment = 0
name_experiment = unique_names_experiments[id_experiment]
print(name_experiment)
where_files = np.where(names_experiments_all==name_experiment)[0]
files_temp = filenames[where_files]

dataset_raw = extractz500_several_files(files_temp)
land_mask_file='/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/LANDFRAC/b.e21.BHISTcmip6.f09_g17.LE2-1001.001.cam.h0.LANDFRAC.185001-185912.nc'
namevar='TREFHTMX'
lsm = xr.open_dataset(land_mask_file).LANDFRAC[0]
lsm = lsm.sel(lat=dataset_raw.lat,lon=dataset_raw.lon)
lsm = lsm.astype(bool)
mask_arr = lsm.data.astype(float)
mask_arr[mask_arr==False]=np.nan
data_arr = dataset_raw[namevar].data
data_arr = data_arr*mask_arr
dataset_raw[namevar].data = data_arr

LE2-1001.001_cmip6


/glade/derecho/scratch/jhayron/tmp/ipykernel_51935/4258214697.py:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetime_index = cftime_index.to_datetimeindex()


In [15]:
dataset_raw['year'] = dataset_raw['time.year']
dataset_raw['dayofyear'] = ('time',np.tile(
        np.arange(1, 366, 1, dtype='int32'),
        len(pd.to_datetime(dataset_raw.time).year.unique())
    ))

In [16]:
maxtemp_reshaped = dataset_raw.set_index(time=('year', 'dayofyear')).unstack('time')

In [18]:
q90 = maxtemp_reshaped.sel(year=slice(1961,1990)).quantile(0.9,dim='year')

/glade/work/jhayron/conda-envs/weather_regimes/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1583: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


In [20]:
stacked = xr.concat([q90, q90, q90], dim='dayofyear')
stacked = stacked.rolling(dayofyear=5, center=True).mean()
q90_smooth = stacked.isel(dayofyear=slice(365, 365*2))

In [27]:
higherthanp90 = maxtemp_reshaped > q90_smooth
higherthanp90 = higherthanp90.stack(time=('year', 'dayofyear'))
higherthanp90 = higherthanp90.transpose('time', 'lat', 'lon').astype(int)

In [34]:
higherthanp90 = higherthanp90.drop_vars(['time','year','dayofyear'])

In [35]:
higherthanp90['time'] = dataset_raw['time']

In [55]:
higherthanp90.to_netcdf(f"{path_outputs_higherthanp90}tmax_higherthanp90_{name_experiment}.nc")

In [50]:
higherthanp90.

<xarray.Dataset> Size: 3GB
Dimensions:   (time: 51100, lat: 64, lon: 121)
Coordinates:
  * lat       (lat) float64 512B 20.26 21.2 22.15 23.09 ... 77.75 78.69 79.63
  * lon       (lon) float64 968B 180.0 181.2 182.5 183.8 ... 327.5 328.8 330.0
    quantile  float64 8B 0.9
  * time      (time) datetime64[ns] 409kB 1961-01-01 1961-01-02 ... 2100-12-31
Data variables:
    TREFHTMX  (time, lat, lon) int64 3GB 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0

'/glade/derecho/scratch/jhayron/Data4WRsClimateChange/LENS2_higherthanp90/tmax_higherthanp90_LE2-1001.001_cmip6.nc'

In [9]:
def process_anoms_and_save(id_experiment):
    name_experiment = unique_names_experiments[id_experiment]
    print(name_experiment)
    where_files = np.where(names_experiments_all==name_experiment)[0]
    files_temp = filenames[where_files]
    
    dataset_raw = extractz500_several_files(files_temp)
    land_mask_file='/glade/campaign/cgd/cesm/CESM2-LE/timeseries/atm/proc/tseries/month_1/LANDFRAC/b.e21.BHISTcmip6.f09_g17.LE2-1001.001.cam.h0.LANDFRAC.185001-185912.nc'
    namevar='TREFHTMX'
    lsm = xr.open_dataset(land_mask_file).LANDFRAC[0]
    lsm = lsm.sel(lat=dataset_raw.lat,lon=dataset_raw.lon)
    lsm = lsm.astype(bool)
    mask_arr = lsm.data.astype(float)
    mask_arr[mask_arr==False]=np.nan
    data_arr = dataset_raw[namevar].data
    data_arr = data_arr*mask_arr
    dataset_raw[namevar].data = data_arr
    
    anoms_std = compute_anomalies(dataset_raw,namevar)
    anoms_std.to_netcdf(f'{path_outputs_anoms}anoms_{name_experiment}.nc')